In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
features = pd.read_csv('../Data/training_set_features.csv')
labels = pd.read_csv('../Data/training_set_labels.csv')

In [3]:
df = pd.merge(features, labels, on='respondent_id', how='inner')

In [4]:
df = df.drop(columns=['employment_occupation', 'employment_industry', 'health_insurance', 'respondent_id'])

In [5]:
plt.show()

In [6]:
seas_df = df.drop(columns=['h1n1_concern',
                          'h1n1_knowledge',
                          'doctor_recc_h1n1',
                          'opinion_h1n1_vacc_effective',
                          'opinion_h1n1_risk',
                          'opinion_h1n1_sick_from_vacc',
                          'h1n1_vaccine'])

h1n1_df = df.drop(columns=['doctor_recc_seasonal',
                          'opinion_seas_vacc_effective',
                          'opinion_seas_risk',
                          'opinion_seas_sick_from_vacc',
                          'seasonal_vaccine'])

In [7]:
categorical_columns = [
    'sex',
    'hhs_geo_region',
    'census_msa',
    'race',
    'age_group',
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'health_worker',
    'child_under_6_months',
    'chronic_med_condition',
    'education',
    'marital_status',
    'employment_status',
    'rent_or_own',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
    'income_poverty'
]

numerical_columns = [
    'household_children',
    'household_adults',
    'h1n1_concern',
    'h1n1_knowledge',
    'opinion_h1n1_risk',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
    
]

for column in categorical_columns:
    curr_col = df[column]
    df.loc[df[column] == 1, column] = 'Yes'
    df.loc[df[column] == 0, column] = 'No'



## Deal with NAs

In [8]:
((df.isnull().sum() / len(df)) * 100).sort_values()

seasonal_vaccine                0.000000
census_msa                      0.000000
hhs_geo_region                  0.000000
sex                             0.000000
race                            0.000000
age_group                       0.000000
h1n1_vaccine                    0.000000
behavioral_face_mask            0.071142
behavioral_wash_hands           0.157262
behavioral_antiviral_meds       0.265848
behavioral_outside_home         0.307036
behavioral_large_gatherings     0.325757
h1n1_concern                    0.344479
h1n1_knowledge                  0.434343
behavioral_touch_face           0.479275
behavioral_avoidance            0.778822
household_children              0.932340
household_adults                0.932340
opinion_h1n1_risk               1.452803
opinion_h1n1_vacc_effective     1.464036
opinion_h1n1_sick_from_vacc     1.479013
opinion_seas_vacc_effective     1.729884
opinion_seas_risk               1.924589
opinion_seas_sick_from_vacc     2.010709
health_worker   

In [9]:
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].mean())

df['health_worker'] = df['health_worker'].fillna(0)
df['behavioral_face_mask'] = df['behavioral_face_mask'].fillna(0)
df['behavioral_wash_hands'] = df['behavioral_wash_hands'].fillna(0)
df['behavioral_antiviral_meds'] = df['behavioral_antiviral_meds'].fillna(0)
df['behavioral_outside_home'] = df['behavioral_outside_home'].fillna(0)
df['behavioral_large_gatherings'] = df['behavioral_large_gatherings'].fillna(0)
df['behavioral_touch_face'] = df['behavioral_touch_face'].fillna(0)
df['behavioral_avoidance'] = df['behavioral_avoidance'].fillna(0)
df['child_under_6_months'] = df['child_under_6_months'].fillna(0)
df['chronic_med_condition'] = df['chronic_med_condition'].fillna(0)
df['marital_status'] = df['marital_status'].fillna('Null')
df['rent_or_own'] = df['rent_or_own'].fillna('Null')
df['education'] = df['education'].fillna('Some College')
df['employment_status'] = df['employment_status'].fillna('Null')
df['doctor_recc_h1n1'] = df['doctor_recc_h1n1'].fillna(1)
df['doctor_recc_seasonal'] = df['doctor_recc_seasonal'].fillna(1)
df['income_poverty'] = df['income_poverty'].fillna('Null')

## Selective Columns

In [10]:
drop_these = [
    'rent_or_own',
    'income_poverty',
    'employment_status',
    'education'
]

categorical_columns = [x for x in categorical_columns if x not in drop_these]

df = df.drop(columns=drop_these)
X = df.drop(columns=['h1n1_vaccine', 'seasonal_vaccine'])
y = df[['h1n1_vaccine', 'seasonal_vaccine']]
y_h1n1 = df[['h1n1_vaccine']]
y_seas = df[['seasonal_vaccine']]

#### Categorical

In [11]:
#Get Binary Data for Categorical Variables
cat_df = X[categorical_columns]

In [12]:
recat_df = pd.get_dummies(data=cat_df)

#### Numerical

In [13]:
num_df = X[numerical_columns]

In [14]:
from sklearn.preprocessing import StandardScaler

#Scale Numerical Data
scaler = StandardScaler()
scaled_num = scaler.fit_transform(num_df)
scaled_num_df = pd.DataFrame(scaled_num, index=num_df.index, columns=num_df.columns)

In [15]:
encoded_df = pd.concat([recat_df, scaled_num_df], axis=1)

In [16]:
encoded_df

,sex_Female,sex_Male,hhs_geo_region_atmpeygn,hhs_geo_region_bhuqouqj,hhs_geo_region_dqpwygqj,hhs_geo_region_fpwskwrf,hhs_geo_region_kbazzjca,hhs_geo_region_lrircsnp,hhs_geo_region_lzgpxyit,hhs_geo_region_mlyzmhmf,...,household_children,household_adults,h1n1_concern,h1n1_knowledge,opinion_h1n1_risk,opinion_h1n1_vacc_effective,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc
0,1,0,0,0,0,0,0,0,0,0,...,-0.578667,-1.182177,-0.680609,-2.046928,-1.052050,-0.850610,-0.264426,-1.880954,-1.253366,-0.089516
1,0,1,0,1,0,0,0,0,0,0,...,-0.578667,-1.182177,1.520279,1.195647,1.298784,1.149360,1.214180,-0.024126,-0.524309,1.426260
2,0,1,0,0,0,0,0,0,0,0,...,-0.578667,1.484891,-0.680609,-0.425641,-1.052050,-0.850610,-1.003729,-0.024126,-1.253366,-0.089516
3,1,0,0,0,0,0,0,1,0,0,...,-0.578667,-1.182177,-0.680609,-0.425641,0.515173,-0.850610,1.953484,0.904289,0.933803,-0.847404
4,1,0,0,0,0,0,0,0,0,0,...,-0.578667,0.151357,0.419835,-0.425641,0.515173,-0.850610,-0.264426,-0.952540,-1.253366,1.426260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26702,1,0,0,0,0,0,0,0,0,0,...,-0.578667,-1.182177,0.419835,-2.046928,-1.052050,-0.850610,-1.003729,0.904289,-0.524309,-0.089516
26703,0,1,0,0,0,0,0,0,1,0,...,-0.578667,0.151357,-0.680609,1.195647,-0.268439,0.149375,-0.264426,0.904289,-1.253366,-0.847404
26704,1,0,0,0,0,0,0,0,1,0,...,-0.578667,-1.182177,0.419835,1.195647,1.298784,0.149375,-0.264426,0.904289,0.933803,-0.089516
26705,1,0,0,0,0,0,0,1,0,0,...,-0.578667,0.151357,-0.680609,-0.425641,-1.052050,-0.850610,-0.264426,-1.880954,-1.253366,-0.089516


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded_df, y, test_size=0.3, random_state=42)

In [18]:
X_train = np.asarray(X_train)
X_test = np.asarray(X_test)
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)
X = np.asarray(encoded_df)

In [19]:
encoded_df.columns

Index(['sex_Female', 'sex_Male', 'hhs_geo_region_atmpeygn',
       'hhs_geo_region_bhuqouqj', 'hhs_geo_region_dqpwygqj',
       'hhs_geo_region_fpwskwrf', 'hhs_geo_region_kbazzjca',
       'hhs_geo_region_lrircsnp', 'hhs_geo_region_lzgpxyit',
       'hhs_geo_region_mlyzmhmf', 'hhs_geo_region_oxchjgsf',
       'hhs_geo_region_qufhixun', 'census_msa_MSA, Not Principle  City',
       'census_msa_MSA, Principle City', 'census_msa_Non-MSA', 'race_Black',
       'race_Hispanic', 'race_Other or Multiple', 'race_White',
       'age_group_18 - 34 Years', 'age_group_35 - 44 Years',
       'age_group_45 - 54 Years', 'age_group_55 - 64 Years',
       'age_group_65+ Years', 'behavioral_face_mask_0',
       'behavioral_face_mask_No', 'behavioral_face_mask_Yes',
       'behavioral_wash_hands_0', 'behavioral_wash_hands_No',
       'behavioral_wash_hands_Yes', 'behavioral_antiviral_meds_0',
       'behavioral_antiviral_meds_No', 'behavioral_antiviral_meds_Yes',
       'behavioral_outside_home_0', 'beha

# Neural Network

In [32]:
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(2, activation='relu', input_dim=73),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(128, activation='tanh'),
    keras.layers.Dense(2, activation='sigmoid')
])

In [33]:
model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [34]:
history = model.fit(
    X_train,
    y_train,
    batch_size=200,
    epochs=5000,
    validation_data=(X_test, y_test)
)

Epoch 1/5000
94/94 [==============================] - 0s 2ms/step - loss: 0.4613 - accuracy: 0.2920 - val_loss: 0.4609 - val_accuracy: 0.2854
Epoch 2/5000
94/94 [==============================] - 0s 1ms/step - loss: 0.4630 - accuracy: 0.2916 - val_loss: 0.4500 - val_accuracy: 0.2854
Epoch 3/5000
94/94 [==============================] - 0s 1ms/step - loss: 0.4560 - accuracy: 0.2916 - val_loss: 0.4469 - val_accuracy: 0.2854
Epoch 4/5000
94/94 [==============================] - 0s 1ms/step - loss: 0.4542 - accuracy: 0.2916 - val_loss: 0.4446 - val_accuracy: 0.2854
Epoch 5/5000
94/94 [==============================] - 0s 1ms/step - loss: 0.4512 - accuracy: 0.2916 - val_loss: 0.4412 - val_accuracy: 0.2854
Epoch 6/5000
94/94 [==============================] - 0s 1ms/step - loss: 0.4483 - accuracy: 0.2916 - val_loss: 0.4403 - val_accuracy: 0.2854
Epoch 7/5000
94/94 [==============================] - 0s 1ms/step - loss: 0.4443 - accuracy: 0.2916 - val_loss: 0.4346 - val_accuracy: 0.2854
Epoch 

KeyboardInterrupt: 

In [35]:
y_true = y
y_predicted = model.predict(X)
y_predicted_binary = np.where(y_predicted > 0.5, 1, 0)

In [36]:
#y_true = y.astype(int)

In [37]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_true, y_predicted, average='macro')

0.7980663899960696

## Submission Data

In [ ]:
test_data = pd.read_csv('../Data/test_set_features.csv')
df_full = test_data

In [ ]:
df = df_full.drop(columns=['employment_occupation', 'employment_industry', 'health_insurance', 'respondent_id'])

categorical_columns = [
    'sex',
    'hhs_geo_region',
    'census_msa',
    'race',
    'age_group',
    'behavioral_face_mask',
    'behavioral_wash_hands',
    'behavioral_antiviral_meds',
    'behavioral_outside_home',
    'behavioral_large_gatherings',
    'behavioral_touch_face',
    'behavioral_avoidance',
    'health_worker',
    'child_under_6_months',
    'chronic_med_condition',
    'education',
    'marital_status',
    'employment_status',
    'rent_or_own',
    'doctor_recc_h1n1',
    'doctor_recc_seasonal',
    'income_poverty'
]

numerical_columns = [
    'household_children',
    'household_adults',
    'h1n1_concern',
    'h1n1_knowledge',
    'opinion_h1n1_risk',
    'opinion_h1n1_vacc_effective',
    'opinion_h1n1_sick_from_vacc',
    'opinion_seas_vacc_effective',
    'opinion_seas_risk',
    'opinion_seas_sick_from_vacc',
    
]

for column in categorical_columns:
    curr_col = df[column]
    df.loc[df[column] == 1, column] = 'Yes'
    df.loc[df[column] == 0, column] = 'No'

In [ ]:
for column in numerical_columns:
    df[column] = df[column].fillna(df[column].mean())

df['health_worker'] = df['health_worker'].fillna(0)
df['behavioral_face_mask'] = df['behavioral_face_mask'].fillna(0)
df['behavioral_wash_hands'] = df['behavioral_wash_hands'].fillna(0)
df['behavioral_antiviral_meds'] = df['behavioral_antiviral_meds'].fillna(0)
df['behavioral_outside_home'] = df['behavioral_outside_home'].fillna(0)
df['behavioral_large_gatherings'] = df['behavioral_large_gatherings'].fillna(0)
df['behavioral_touch_face'] = df['behavioral_touch_face'].fillna(0)
df['behavioral_avoidance'] = df['behavioral_avoidance'].fillna(0)
df['child_under_6_months'] = df['child_under_6_months'].fillna(0)
df['chronic_med_condition'] = df['chronic_med_condition'].fillna(0)
df['marital_status'] = df['marital_status'].fillna('Not Married')
df['rent_or_own'] = df['rent_or_own'].fillna('Rent')
df['education'] = df['education'].fillna('Some College')
df['employment_status'] = df['employment_status'].fillna('Employed')
df['doctor_recc_h1n1'] = df['doctor_recc_h1n1'].fillna(1)
df['doctor_recc_seasonal'] = df['doctor_recc_seasonal'].fillna(1)
df['income_poverty'] = df['income_poverty'].fillna('<= $75,000, Above Poverty')

In [ ]:
X = df

drop_these = [
    'rent_or_own',
    'income_poverty',
    'employment_status',
    'education'
]

categorical_columns = [x for x in categorical_columns if x not in drop_these]

df = df.drop(columns=drop_these)


#Get Binary Data for Categorical Variables
cat_df = X[categorical_columns]
recat_df = pd.get_dummies(data=cat_df)

num_df = X[numerical_columns]

from sklearn.preprocessing import StandardScaler

#Scale Numerical Data
scaler = StandardScaler()
scaled_num = scaler.fit_transform(num_df)
scaled_num_df = pd.DataFrame(scaled_num, index=num_df.index, columns=num_df.columns)

encoded_df = pd.concat([recat_df, scaled_num_df], axis=1)

X = np.asarray(encoded_df)

In [ ]:
y = model.predict(X)
y_df = pd.DataFrame(y, columns=['h1n1_vaccine', 'seasonal_vaccine'])

In [ ]:
results = pd.concat([df_full, y_df], axis=1)

In [ ]:
results = results[['respondent_id', 'h1n1_vaccine', 'seasonal_vaccine']]

In [ ]:
results.to_csv('../Submissions/Submission 6.6.21.csv', index=False)